In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
data = pd.read_csv('assets/wave_height_hourly.csv')

In [ ]:
data.head()

In [ ]:
wave = pd.Series(data['SignificantWaveHeight'].values, index = pd.to_datetime(data['time']))
wave.plot()

In [ ]:
from src.tde import UnivariateTDE

In [ ]:
# prever os valores das próximas 12 horas com base nas últimas 6
wave_tde = UnivariateTDE(wave, horizon=12, k=6)
wave_tde.head()

In [ ]:
is_future = wave_tde.columns.str.contains('\+')
X = wave_tde.iloc[:,~is_future]
Y = wave_tde.iloc[:,is_future]

X.head()

In [ ]:
Y.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

### Método Direto

In [ ]:
# Abordagem direta
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

?MultiOutputRegressor

In [ ]:
model = MultiOutputRegressor(Ridge())

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
forecasts_direto = model.predict(X_test)
print(forecasts_direto[:5])
forecasts_direto = pd.DataFrame(forecasts_direto, columns = Y_test.columns)
forecasts_direto.head()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
error_by_horizon_dir = {}
for h in Y_test.columns:
    error_by_horizon_dir[h] = mean_absolute_error(Y_test[h], forecasts_direto[h])
    
error_by_horizon_dir

In [ ]:
pd.Series(error_by_horizon_dir).plot()

### Método Recursivo

In [ ]:
model = Ridge()
print(Y_train['t+1'].head())
model.fit(X_train, Y_train['t+1'])

In [ ]:
Y_hat = pd.DataFrame(np.zeros((X_test.shape[0], 12)), columns=Y_train.columns)
Y_hat.head()

In [ ]:
yh = model.predict(X_test)
Y_hat['t+1'] = yh

X_ = X_test.copy()
for i in range(2, 12 + 1):
    print('Antes')
    print(X_.head(3))
    X_.iloc[:, :-1] = X_.iloc[:, 1:].values
    X_['t-0'] = yh
    print('Depois')
    print(X_.head(3))

    yh = model.predict(X_)

    Y_hat[f't+{i}'] = yh

In [ ]:
Y_hat.head()

In [ ]:
error_by_horizon_rec = {}
for h in Y_test.columns:
    error_by_horizon_rec[h] = mean_absolute_error(Y_test[h], Y_hat[h])
    
error_by_horizon_rec

In [ ]:
pd.Series(error_by_horizon_rec).mean()

In [ ]:
pd.Series(error_by_horizon_dir).mean()

### Método Direto-Recursivo (DirRec)

In [ ]:
from sklearn.multioutput import RegressorChain

?RegressorChain

model = RegressorChain(Ridge())
model.fit(X_train, Y_train)

In [ ]:
forecasts_dr = model.predict(X_test)
forecasts_dr = pd.DataFrame(forecasts_dr, columns = Y_test.columns)
forecasts_dr.head()

In [ ]:
error_by_horizon_dr = {}
for h in Y_test.columns:
    error_by_horizon_dr[h] = mean_absolute_error(Y_test[h], forecasts_dr[h])
    
error_by_horizon_dr

In [ ]:
# pd.Series(error_by_horizon_dr)
pd.Series(error_by_horizon_dr).mean()

### Método Multi-output

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model_mo = KNeighborsRegressor()
model_mo.fit(X_train, Y_train)

In [ ]:
forecasts_mo = model_mo.predict(X_test)
forecasts_mo = pd.DataFrame(forecasts_mo, columns = Y_test.columns)
forecasts_mo.head()

In [ ]:
error_by_horizon_mo = {}
for h in Y_test.columns:
    error_by_horizon_mo[h] = mean_absolute_error(Y_test[h], forecasts_mo[h])
    
error_by_horizon_mo

In [ ]:
pd.Series(error_by_horizon_mo).mean()

In [ ]:
print(f'Erro Direto: {pd.Series(error_by_horizon_dir).mean()}')
print(f'Erro Recursivo: {pd.Series(error_by_horizon_rec).mean()}')
print(f'Erro Direto-Recursivo: {pd.Series(error_by_horizon_dr).mean()}')
print(f'Erro Multi-output: {pd.Series(error_by_horizon_mo).mean()}')